# CACTUSTORCH Remote Thread Creation
Detects remote thread creation from CACTUSTORCH as described in references.

## Rule Content
```
- title: CACTUSTORCH Remote Thread Creation
  id: 2e4e488a-6164-4811-9ea1-f960c7359c40
  description: Detects remote thread creation from CACTUSTORCH as described in references.
  references:
  - https://twitter.com/SBousseaden/status/1090588499517079552
  - https://github.com/mdsecactivebreach/CACTUSTORCH
  status: experimental
  author: '@SBousseaden (detection), Thomas Patzke (rule)'
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 8
      SourceImage:
      - '*\System32\cscript.exe'
      - '*\System32\wscript.exe'
      - '*\System32\mshta.exe'
      - '*\winword.exe'
      - '*\excel.exe'
      TargetImage: '*\SysWOW64\\*'
      StartModule: null
    condition: selection
  tags:
  - attack.execution
  - attack.t1055
  - attack.t1064
  falsepositives:
  - unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"8" AND process_path.keyword:(*\\System32\\cscript.exe OR *\\System32\\wscript.exe OR *\\System32\\mshta.exe OR *\\winword.exe OR *\\excel.exe) AND target_process_path.keyword:*\\SysWOW64\\* AND NOT _exists_:thread_start_module)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()